# Morphology Module: Part 1 for CNS tutorial
----

*Load morphologies that were downloaded, analyze them, and compare species.*

For the first part of the tutorial, the user needs to create a *virtual lab* and save the URL of the lab. This will be used to authenticate the access of the user and allow access to the data. Once authentication is verified, the user will be able to access the data and search through it. We will select data by their species. We will download 20 mouse morphologies, 20 rat morphologies and 20 human morphologies from the temporal cortex. We save these neurons locally in different folders. 

## 1.1 Authentication process

In [ ]:
from obi_auth import get_token
from obi_notebook.get_projects import get_projects

access_token = get_token(environment="production", auth_mode="daf")

In [ ]:
from uuid import UUID
from entitysdk.client import Client
from entitysdk.common import ProjectContext
from entitysdk.models import (
    BrainLocation,
    BrainRegion,
    Species,
    Strain,
    MTypeClass,
)
try:
    from entitysdk.models import ReconstructionMorphology
except:
    from entitysdk.models import CellMorphology as ReconstructionMorphology

from rich import print as rprint

In [ ]:
from obi_notebook.get_projects import get_projects

project_context = get_projects(token=access_token)

In [ ]:
client = Client(project_context=project_context,
                environment="production",
                token_manager=access_token)

## 1.2 Explore all species, brain regions and mtypes

Here we will explore the available data in the database to identify species and regions of interest. 

**Species**

In [ ]:
all_species = client.search_entity(entity_type=Species).all()
rprint(all_species)

**Brain regions**

In [ ]:
all_regions = client.search_entity(entity_type=BrainRegion).all()
rprint(all_regions[:2])

**Find regions for temporal cortex for example**

In [ ]:
rprint([i.name for i in all_regions if "Temporal" in i.name ])

**Mtypes**

In [ ]:
all_mtypes = client.search_entity(entity_type=MTypeClass).all()
rprint([i.pref_label for i in all_mtypes[:10]])

## 1.3 Find morphologies of a specific mtype: L3_TPC:A

We select an mtype and we find a morphology based on our query. 

In [ ]:
iterator = client.search_entity(
    entity_type=ReconstructionMorphology,
    query={"mtype__pref_label": "L3_TPC:A"},
    limit=1,
)

Check the properties of a selected morphology (we selected only one example here)

In [ ]:
morphology = next(iterator)

Mtype of the moprphology is the morphological cell type assigned by a classification scheme

In [ ]:
rprint(morphology)

## 1.4 Find 10 morphologies of specific mtype, species and region

An example of findinding morphologies based on a combination of filters:

* Species
* Mtype
* Brain region

In [ ]:
iterator = client.search_entity(
    entity_type=ReconstructionMorphology,
    query={"mtype__pref_label": "L3_TPC:A", "species__name":"Mus musculus", "brain_region__name":"Primary somatosensory area"},
    limit=10,
)

In [ ]:
morphology = next(iterator)

## 1.5 Explore the metadata of a morphology

For one morphology we explore the available metadata that may be important for our analysis. 

In [ ]:
morphology.species.name

In [ ]:
morphology.brain_region.name

In [ ]:
morphology.id

In [ ]:
morphology.mtypes[0].pref_label

In [ ]:
for asset in morphology.assets:
    print(asset.path)

## 1.6 Download a morphology file

Now we need to download an example morphology. Even though we access the metadata, getting the "asset", i.e. the file, of the morphology needs a different process.

In [ ]:
fetched = client.get_entity(
    entity_id=morphology.id, entity_type=ReconstructionMorphology
)

In [ ]:
for asset in fetched.assets:
    if asset.content_type == "application/swc":
        client.download_file(
            entity_id=fetched.id,
            entity_type=type(fetched),
            asset_id=asset.id,
            output_path="test_data",
        )
        content = client.download_content(
            entity_id=fetched.id, entity_type=type(fetched), asset_id=asset.id
        )
        break

In [ ]:
ls test_data/

## 1.7 Download 20 morphologies for each species

For now we perform comparison of mouse / rat from Primary somatosensory area, due to lack of appropriate data in entitycore. 
This will change to mouse / human from temporal association areas.

**Mouse**

In [ ]:
iterator = client.search_entity(
    entity_type=ReconstructionMorphology,
    query={"mtype__pref_label": "L3_TPC:A", "species__name":"Mus musculus", "brain_region__name":"Primary somatosensory area"},
    limit=20,
)

In [ ]:
rm -r Mouse

In [ ]:
mkdir Mouse

In [ ]:
for morphology in iterator:
    fetched = client.get_entity(
        entity_id=morphology.id, entity_type=ReconstructionMorphology
        )
    for asset in fetched.assets:
        if asset.content_type == "application/swc":
            client.download_file(
                entity_id=fetched.id,
                entity_type=type(fetched),
                asset_id=asset.id,
                output_path="Mouse/",
            )
            content = client.download_content(
                entity_id=fetched.id, entity_type=type(fetched), asset_id=asset.id
            )
            break

In [ ]:
ls Mouse

**Rat**

In [ ]:
iterator = client.search_entity(
    entity_type=ReconstructionMorphology,
    query={"mtype__pref_label": "L3_TPC:A", "species__name":"Rattus norvegicus"},# "brain_region__name":"Primary somatosensory area"},
    limit=20,
)

In [ ]:
rm -r Rat

In [ ]:
mkdir Rat

In [ ]:
for morphology in iterator:
    fetched = client.get_entity(
        entity_id=morphology.id, entity_type=ReconstructionMorphology
        )
    for asset in fetched.assets:
        if asset.content_type == "application/swc":
            client.download_file(
                entity_id=fetched.id,
                entity_type=type(fetched),
                asset_id=asset.id,
                output_path="Rat/",
            )
            content = client.download_content(
                entity_id=fetched.id, entity_type=type(fetched), asset_id=asset.id
            )
            break

In [ ]:
ls Rat

**Human**

In [ ]:
iterator = client.search_entity(
    entity_type=ReconstructionMorphology,
    query={"contribution__pref_label": "Christiaan de Kock", "mtype__pref_label": "L23_PC", "species__name":"Homo sapiens"}, #"brain_region__name":"Temporal association areas, layer 2/3"},    
    limit=20,
)

In [ ]:
rm -r Human

In [ ]:
mkdir Human

In [ ]:
for morphology in iterator:
    fetched = client.get_entity(
        entity_id=morphology.id, entity_type=ReconstructionMorphology
        )
    for asset in fetched.assets:
        if asset.content_type == "application/swc":
            client.download_file(
                entity_id=fetched.id,
                entity_type=type(fetched),
                asset_id=asset.id,
                output_path="Human/",
            )
            content = client.download_content(
                entity_id=fetched.id, entity_type=type(fetched), asset_id=asset.id
            )
            break

In [ ]:
ls Human